# Week 2 - Django Intro

OK, we are going to spend the first week working with Django's ORM and building models.

But first, we need to start a Django project. How do we do that?

## 1. Setting up the environment

First, set up the workspace.
1. Create a project directory and move into it.
2. Initialize a git repository for the project.
3. Create a Python virtual environment
4. Create a requirements file

```bash
$ mkdir mylibrary
$ cd mylibrary
$ git init
Initialized empty Git repository in /Users/jgomez/projects/mylibrary/.git/
$ pyvenv ENV
$ source ENV/bin/activate
(ENV)$ vim requirements.txt
```

Put your dependencies in the `requirements.txt` file. In this case, we only have one dependency: Django. But I also recommend installing iPython.

```
Django
ipython
```

Now install the dependencies with `pip`

```bash
(ENV)$ pip install -r requirements
Collecting Django (from -r requirements.txt (line 1))
  Using cached Django-1.11-py2.py3-none-any.whl
Collecting pytz (from Django->-r requirements.txt (line 1))
  Using cached pytz-2017.2-py2.py3-none-any.whl
Installing collected packages: pytz, Django
Successfully installed Django-1.11 pytz-2017.2
```

## 2. Starting a project

OK, our environment is ready, now let's start the Django project.

```bash
(ENV)$ django-admin startproject mylibrary
(ENV)$ ls
ENV			mylibrary		requirements.txt
$ cd mylibrary/
(ENV)$ ls
manage.py	mylibrary
(ENV)$ ls mylibrary/
__init__.py	settings.py	urls.py		wsgi.py
```

You can see that the this has created a directory structure for you. Your project now looks like this:

```
mylibrary/
  |
  |- ENV/
  |- .git/
  |- requirements.txt
  |- mylibrary/
      |
      |- manage.py
      |- mylibrary/
           |- __init__.py
           |- settings.py
           |- urls.py
           |- wsgi.py
```

At the top level, you have a `mylibrary` directory, which is your git repo.
Inside your git repo you have another `mylibrary` directory, which is the root of your Django project. It contains the `manage.py` file, which is used to perform other Django management commands.
Inside your Django root folder, you have yet another `mylibrary` directory, which has configuration files for your Django project. These include:
- `settings.py` which contains lots of settings for Django, your database, and your applications
- `urls.py` which you will use to create the URL patterns for your applications
- `wsgi.py` which is a special file that acts as a bridge between a production web server (e.g. Apache) and your Python code.

A Django *project* is mostly just administrative overhead. The real logic lives in Django *applications*. So let's create one of those.

```bash
(ENV)$ python manage.py startapp collection
(ENV)$ ls
collection	manage.py	mylibrary
(ENV)$ ls collection/
__init__.py	admin.py	apps.py		migrations	models.py	tests.py	views.py
```

OK, so we now have a `collection` directory under our Django root, with lots of empty Python files in it.

```
mylibrary/
  |- ENV/
  |- .git/
  |- requirements.txt
  |- mylibrary/
      |- manage.py
      |- mylibrary/
           |- __init__.py
           |- settings.py
           |- urls.py
           |- wsgi.py
      |- collection/
           |- __init__.py
           |- admin.py
           |- apps.py
           |- migrations/
           |- models.py
           |- tests.py
           |- views.py
```

There's a lot in here. This week we will focus on the `models.py` file and the `migrations` folder.

## 3. Creating Models

Let's open up the `models.py file and create a really basic model for books in our collection. *Remember* our models must be subclasses of the Django `Model` class. Let's start off with just two fields for our Book model: title and ISBN.

```python
# models.py
from django.db import models


class Book(models.Model):

	title = models.CharField(max_length=256)
	isbn = models.CharField(max_length=13)

```

OK, now let's try interacting with out model. We can use Django's shell instead of the default Python shell. This makes it aware of our Django project for easier importing.

```bash
(ENV)$ python manage shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```

```python
>>> from collection.models import Book
Traceback (most recent call last):
  File "<console>", line 1, in <module>
  File "/Users/jgomez/projects/mylibrary/mylibrary/collection/models.py", line 4, in <module>
    class Book(models.Model):
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 118, in __new__
    "INSTALLED_APPS." % (module, name)
RuntimeError: Model class collection.models.Book doesn't declare an explicit app_label and isn't in an application in INSTALLED_APPS.
```

Oh no! What happened? Our Django project is not aware of our application. We have to add it to the `INSTALLED_APPS` list in the `settings.py` file.

```python
# settings.py
# ...
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'collection',   # <-- add this line!
]
```

OK, let's try that again.

```bash
(ENV)$ python manage.py shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```

```python
>>> from collection.models import Book
>>> b = Book()
>>> b.title = "Cat's Cradle"
>>> b.isbn = '1234567890'
>>> b.save()
Traceback (most recent call last):
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/sqlite3/base.py", line 328, in execute
    return Database.Cursor.execute(self, query, params)
sqlite3.OperationalError: no such table: collection_book

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<console>", line 1, in <module>
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 806, in save
    force_update=force_update, update_fields=update_fields)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 836, in save_base
    updated = self._save_table(raw, cls, force_insert, force_update, using, update_fields)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 922, in _save_table
    result = self._do_insert(cls._base_manager, using, fields, update_pk, raw)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 961, in _do_insert
    using=using, raw=raw)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/manager.py", line 85, in manager_method
    return getattr(self.get_queryset(), name)(*args, **kwargs)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/query.py", line 1060, in _insert
    return query.get_compiler(using=using).execute_sql(return_id)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/sql/compiler.py", line 1099, in execute_sql
    cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 80, in execute
    return super(CursorDebugWrapper, self).execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/utils.py", line 94, in __exit__
    six.reraise(dj_exc_type, dj_exc_value, traceback)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/utils/six.py", line 685, in reraise
    raise value.with_traceback(tb)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/sqlite3/base.py", line 328, in execute
    return Database.Cursor.execute(self, query, params)
django.db.utils.OperationalError: no such table: collection_book
>>> 
```

Oh no again! Now what happened? It's telling us there is no table in the database to store our books. That's because we never actually told Django to push our new model definition to the database. How do we do that? We ***migrate***.

Take a look inside the `migrations` directory.

```bash
(ENV)$ ls collection/migrations/
__init__.py
```

There's nothing in there except the `__init__.py` file, which is just a required file for a directory to be a Python package.

So let's try *migrating* our database.

```bash
(ENV)$ python manage.py makemigrations
Migrations for 'collection':
  collection/migrations/0001_initial.py
    - Create model Book
```

OK, you can see that it created a file in our migrations folder. Let's open it up and see what's in it.

```python
# collection/migrations/0001_initial.py

# -*- coding: utf-8 -*-
# Generated by Django 1.11 on 2017-04-14 18:14
from __future__ import unicode_literals

from django.db import migrations, models


class Migration(migrations.Migration):

    initial = True

    dependencies = [
    ]

    operations = [
        migrations.CreateModel(
            name='Book',
            fields=[
                ('id', models.AutoField(auto_created=True, primary_key=True, serialize=False, verbose_name='ID')),
                ('title', models.CharField(max_length=256)),
                ('isbn', models.CharField(max_length=13)),
            ],
        ),
    ]
```

You can see that it is some Python code used by Django. It tells Django which operations it needs to perform on the database to bring it up to date with our models. In this case, it needs to simply *create* a Model table, with *3* field. Yes, it is adding an `id` field, that we did not specify. This is the primary key field that is autogenerated, unless we override it.

We are not yet done with the migration! We have simply defined the migration that is necessary. Now we have to execute it.

```bash
(ENV)$ python manage.py migrate
Operations to perform:
  Apply all migrations: admin, auth, collection, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying collection.0001_initial... OK
  Applying sessions.0001_initial... OK
 ```
 
You can see that in addition to our `collection` application, Django has applied the migrations for other built-in apps, used for site administration, authentication, and session management.

OK, now let's go back to our shell and try saving an object again.

```bash
(ENV)$ python manage.py shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```

```python
>>> from collection.models import Book
>>> b1 = Book(title="Cats Cradle", isbn='1234567890')
>>> b1.save()
>>> b2 = Book()
>>> b2.title = "Slaughterhouse Five"
>>> b2.isbn = '0987654321'
>>> b2.save()
>>> 
```

Great, it worked! Let's double check. You can see that we now have a SQLite database in our Django root directory.

```bash
(ENV)$ ls
collection	db.sqlite3	manage.py	mylibrary
```

To use the databse we could call `sqlite3 db.sqlite3`. However, once we move to production databases, like MySQL or PostgreSQL, you will also need usernames and password. Django provides an easy way to login to the database, with the `dbshell` command.

```bash
(ENV)$ python manage.py dbshell
SQLite version 3.8.10.2 2015-05-20 18:17:19
Enter ".help" for usage hints.
sqlite> 
```

Let's take a look at our tables, check the SQL that Django used to create our book table, and then see what's in it.

```sql
sqlite> .tables
auth_group                  collection_book           
auth_group_permissions      django_admin_log          
auth_permission             django_content_type       
auth_user                   django_migrations         
auth_user_groups            django_session            
auth_user_user_permissions

sqlite> .schema collection_book
CREATE TABLE "collection_book" ("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "title" varchar(256) NOT NULL, "isbn" varchar(13) NOT NULL);

sqlite> select * from collection_book;
1|Cats Cradle|1234567890
2|Slaughterhouse Five|0987654321
sqlite> 
```

OK, so there is our proof that the Django ORM is working. It turned out Model class definition into Data Definition Language syntax to create our table, and when we called `save()` on our Python objects, it sent the appropriate SQL insert statements to the database.

What happens if we decide to change out model? For instance, we now want to add the year of publication to our books. Let's update our model...

```python
# models.py
from django.db import models


class Book(models.Model):

	title = models.CharField(max_length=256)
	isbn = models.CharField(max_length=13)
    pub_year = models.IntegerField()
```

Now let's fire up the shell again and save a new book.

```bash
(ENV)$ python manage.py shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```

```python
>>> from collection.models import Book
>>> b3 = Book(title='Hocus Pocus', isbn='1111111111', pub_year=1996)
>>> b3.save()
Traceback (most recent call last):
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/sqlite3/base.py", line 328, in execute
    return Database.Cursor.execute(self, query, params)
sqlite3.OperationalError: table collection_book has no column named pub_year

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<console>", line 1, in <module>
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 806, in save
    force_update=force_update, update_fields=update_fields)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 836, in save_base
    updated = self._save_table(raw, cls, force_insert, force_update, using, update_fields)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 922, in _save_table
    result = self._do_insert(cls._base_manager, using, fields, update_pk, raw)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/base.py", line 961, in _do_insert
    using=using, raw=raw)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/manager.py", line 85, in manager_method
    return getattr(self.get_queryset(), name)(*args, **kwargs)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/query.py", line 1060, in _insert
    return query.get_compiler(using=using).execute_sql(return_id)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/sql/compiler.py", line 1099, in execute_sql
    cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 80, in execute
    return super(CursorDebugWrapper, self).execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/utils.py", line 94, in __exit__
    six.reraise(dj_exc_type, dj_exc_value, traceback)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/utils/six.py", line 685, in reraise
    raise value.with_traceback(tb)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/utils.py", line 65, in execute
    return self.cursor.execute(sql, params)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/backends/sqlite3/base.py", line 328, in execute
    return Database.Cursor.execute(self, query, params)
django.db.utils.OperationalError: table collection_book has no column named pub_year
>>> 
```

Great, now what happened?! Look at the last line of the error statement. It is telling us that there is no column for `pub_year`. The table never got update because we didn't migrate! Let's fix that.

```bash
(ENV)$ python manage.py makemigrations
You are trying to add a non-nullable field 'pub_year' to book without a default; we can't do that (the database needs something to populate existing rows).
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Quit, and let me add a default in models.py
Select an option: 2
```

Whoa! What's this? All model fields are *required* unless we specify that `null=True`. But if we already have two rows in the database without a `pub_year` value, then how can the database be consistent? It requires that we specify a value for those old rows. So, our option is to specify a default value right here in the `makemigrations` command, or we could go back to the model and add a `default="some value"` option the field. A third option is to allow null values in that column. Some people say that null values are bad, and they have a good point. But what default value could I possibly give for `pub_year` that would make sense? 0? But some books really were created in year 0! So let's just allow it to be null. Select option 2, then edit your `models.py` file.

```python
# models.py
from django.db import models


class Book(models.Model):

    title = models.CharField(max_length=256)
    isbn = models.CharField(max_length=13)
    pub_year = models.IntegerField(null=True)
```

OK, now let's try this migration again.

```bash
(ENV)$ python manage.py makemigrations
Migrations for 'collection':
  collection/migrations/0002_book_pub_year.py
    - Add field pub_year to book
(ENV)$ python manage.py migrate
Operations to perform:
  Apply all migrations: admin, auth, collection, contenttypes, sessions
Running migrations:
  Applying collection.0002_book_pub_year... OK
```

Now we can go back to the shell and create a book with a pub year value.

```bash
$ python manage.py shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```

```python
>>> from collection.models import Book
>>> b3 = Book(title="Hocus Pocus", isbn="1122334455", pub_year=1996)
>>> b3.save()
>>> 
```

## 4. Querying models

OK, now let's try getting stuff back out of the database. Each model class has an attribute called `objects`. This is the model's Manager object. You use it to select a set of objects (rows) from the database.

```python
>>> Book.objects
<django.db.models.manager.Manager object at 0x104918a90>
```

### Single item retrieval

You can call the `.get()` method on the Manager to get a specific book. Specify that you want the book with primary key of 1.

```python
>>> book1 = Book.objects.get(pk=1)
>>> book1.title
"Cats Cradle"
>>> 
```

The `.get()` method only works when your query results in a single object. If multiple objects match the query parameters, then it will throw an exception.

```python
>>> book1 = Book.objects.get()
Traceback (most recent call last):
  File "<console>", line 1, in <module>
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/manager.py", line 85, in manager_method
    return getattr(self.get_queryset(), name)(*args, **kwargs)
  File "/Users/jgomez/projects/mylibrary/ENV/lib/python3.5/site-packages/django/db/models/query.py", line 383, in get
    (self.model._meta.object_name, num)
collection.models.MultipleObjectsReturned: get() returned more than one Book -- it returned 3!
```

### Retrieving sets and filtering
If you want more than a single object, you can use the `all()` method to get everything. That will return a QuerySet object. You can then call multiple filters on the QuerySet to refine the result set.

```python
>>> books = Book.objects.all()
>>> books
<QuerySet [<Book: Book object>, <Book: Book object>, <Book: Book object>]>
>>> 
>>> for book in books:
...     print(book.title)
... 
Cats Cradle
Slaughterhouse Five
Hocus Pocus
>>> 
>>> modern_books = Book.objects.all().filter(pub_year__gte=1990)
>>> for book in modern_books:
...     print(book.title)
... 
Hocus Pocus
>>> 
```

We used a filter above to get only books published during or after 1990. To do that we passed in `pub_year` to the filter, but we altered it by adding a double underscore and `gte`. This tells Django we want books where the `pub_year` is greater than or equal to 1990. There are lots of similar parameters you can pass to the filter. We won't go through them here. You should read the documentation to get familiar with them.

### Overriding the __repr__ method
When we list the books in the QuerySet, we don't get very useful information. They all say `<Book: Book object>`. Let's override the `__repr__()` method of Book, to give us better info.


```python
# models.py
from django.db import models


class Book(models.Model):

    title = models.CharField(max_length=256)
    isbn = models.CharField(max_length=13)
    pub_year = models.IntegerField(null=True)

    def __repr__(self):
        return '<Book: {}>'.format(self.title)
```

Now let's reload the shell to get the new version of our code.

```bash
(ENV) GT26909:mylibrary jgomez$ python manage.py shell
Python 3.5.1 (v3.5.1:37a07cee5969, Dec  5 2015, 21:12:44) 
[GCC 4.2.1 (Apple Inc. build 5666) (dot 3)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
(InteractiveConsole)
```
```python
>>> from collection.models import Book
>>> Book.objects.all()
<QuerySet [<Book: Cats Cradle>, <Book: Slaughterhouse Five>, <Book: Hocus Pocus>]>
```